In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [2]:

train_path = '/kaggle/input/plant-pathology-2020-fgvc7/train.csv'
test_path = '/kaggle/input/plant-pathology-2020-fgvc7/test.csv'
sample_submission_path = '/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv'   

data= pd.read_csv(train_path)
data_test = pd.read_csv(test_path)

In [3]:
# data.apply(lambda row : row['image_id']=='Train_0'.in, axis =1)
def labbeling_df(image_id):
    index= 0
    for i in data[data.image_id==image_id].values[0]:
        index += 1
        if i ==1:
            return(data.columns[index-1])
        else:
            pass


In [4]:
data['label'] = data.apply(lambda x : labbeling_df(x.image_id),axis=1)
images_arr = os.listdir('/kaggle/input/plant-pathology-2020-fgvc7/images')
data.head(10)

In [6]:
from keras.preprocessing.image import ImageDataGenerator
# Images Augmentation
rotate = ImageDataGenerator(rotation_range=30-80) # rotation
hor_shift = ImageDataGenerator(width_shift_range=0.2)  # horizontal shift
zoom = ImageDataGenerator(zoom_range=[0.7,0.8])  # zoom
hor_flip = ImageDataGenerator(horizontal_flip=True)  # horizontal flip
brightness = ImageDataGenerator(brightness_range = [0.4,0.5])                                # brightness

In [7]:
# Adding more Augmented images so as to increase the data size
import cv2
from tqdm import tqdm
train_images = []
all_labels = []

for id_ in tqdm(data.image_id):
    img = cv2.imread(f'../input/plant-pathology-2020-fgvc7/images/{id_}.jpg')
    img = cv2.resize(img,(100,100),interpolation=cv2.INTER_AREA)
    img_feature = np.ndarray(shape = (1, 100, 100, 3),dtype = np.float32)
    img_feature[0] = img
    
    img_iterator_zoom = zoom.flow(img_feature)
    img_iterator_rotate = rotate.flow(img_feature)
    img_iterator_hor_shift = hor_shift.flow(img_feature)
    img_iterator_hor_flip = hor_flip.flow(img_feature)
    img_iterator_brightness = brightness.flow(img_feature)
    
    zoom_image  = img_iterator_zoom[0].astype('uint8')
    rotated_image  = img_iterator_rotate[0].astype('uint8')
    hor_shift_image  = img_iterator_hor_shift[0].astype('uint8')
    hor_flip_image  = img_iterator_hor_flip[0].astype('uint8')
    bright_image  = img_iterator_brightness[0].astype('uint8')
    
    train_images.append(img_feature)
    train_images.extend([zoom_image, rotated_image, hor_shift_image, hor_flip_image, bright_image])
    
    label_array = data.loc[data.image_id ==id_,['healthy','multiple_diseases','rust','scab']].values[0]
    label_list = list(label_array)
    all_labels.extend([label_list for i in range(0,6)])

    
    


In [8]:
# Check the data size
len(all_labels), len(train_images)

In [9]:
# Assigning train images as array
feature = np.ndarray(shape = (len(train_images), 100, 100, 3),dtype = np.float32)
for i in range(len(train_images)):
    feature[i] = train_images[i]


feature = feature/255
# Getting labels in the array
labels = np.array(all_labels)

In [ ]:
# Loadig test data
test_images = []
for id_ in data_test.image_id:
    img = cv2.imread(f'../input/plant-pathology-2020-fgvc7/images/{id_}.jpg')
    img = cv2.resize(img,(100,100),interpolation=cv2.INTER_AREA)
    test_images.append(img)
    
x_test = np.ndarray(shape = (len(test_images), 100, 100, 3),dtype = np.float32)

for i in range(len(test_images)):
    x_test[i] = test_images[i]

x_test = x_test / 255

In [11]:

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(feature, labels, test_size = 0.2)

In [17]:
# For CNN model to train to classify the images
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization
# from keras.utils import np_utils

# building a linear stack of layers with the sequential model
model = Sequential()

# convolutional layer 1
model.add(Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.10))

# convolutional layer 2
model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.10))

# Convolutional layer 3
model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.10))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# output layer
model.add(Dense(4, activation='softmax'))
"""

model = Sequential()

# Layer 1: Input  
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', input_shape=(100, 100, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Layer 2  
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
   
# Layer 3
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(5, 5), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Layer 4
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=256, kernel_size=(5, 5), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Layer 5: Output
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

"""

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_val, y_val))


# training the model for 10 epochs
# history = model.fit(train_datagen.flow(x_train, y_train, batch_size=32), epochs=20, validation_data=(x_val, y_val))


In [18]:
import matplotlib.pyplot as plt
plt.figure(figsize=(13,8))
plt.plot(history.history['val_accuracy'], label ='Validation Accuracy')
plt.plot(history.history['accuracy'], label = 'Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Loadig test data
test_images = []
for id_ in data_test.image_id:
    img = cv2.imread(f'../input/plant-pathology-2020-fgvc7/images/{id_}.jpg')
    img = cv2.resize(img,(100,100),interpolation=cv2.INTER_AREA)
    test_images.append(img)
    
x_test = np.ndarray(shape = (len(test_images), 100, 100, 3),dtype = np.float32)

for i in range(len(test_images)):
    x_test[i] = test_images[i]

x_test = x_test / 255

In [ ]:
predicted_ = model.predict(x_test)
predicted_test_df = pd.DataFrame(predicted_, columns =['healty','multiple_disease','rust','scab'])


In [ ]:
# x_train.shape
#sklearn expects i/p to be 2d array-model.fit(x_train,y_train)=>reshape to 2d array
nsamples, nx, ny, nrgb = x_train.shape
x_train_sklearn = x_train.reshape((nsamples,nx*ny*nrgb))

#so,eventually,model.predict() should also be a 2d input
nsamples, nx, ny, nrgb = x_val.shape
x_val_sklearn = x_val.reshape((nsamples,nx*ny*nrgb))

In [ ]:
# from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_ = le.fit_transform(y_train)

In [ ]:
# Applying Random Forest algorithms 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model=RandomForestClassifier()
forest_params = [{'max_depth': list(range(10, 15)), 'max_features': list(range(1000,1100))}]

clf = GridSearchCV(model, forest_params, cv = 10, scoring='accuracy')


clf.fit(x_train_sklearn,y_train)

In [ ]:
clf

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

def model_performance_comparision(X_train, y_train):
    # Definfing models
    models = [
#         RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
        LinearSVC(),
        MultinomialNB(),
#         LogisticRegression(random_state=0),
    ]
    # 5 Cross-validation
    CV = 3
    cv_df = pd.DataFrame(index=range(CV * len(models)))

    entries = []
    for model in models:
        model_name = model.__class__.__name__
        print(model_name + "started")
        accuracies = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=CV)
        for fold_idx, accuracy in enumerate(accuracies):
            entries.append((model_name, fold_idx, accuracy))
        cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
    return(cv_df)

In [ ]:
model_performace_df = model_performance_comparision(x_train_sklearn,y_train_)

In [ ]:
model_performace_df

In [ ]:


y_pred=model.predict(x_val_sklearn)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np
accuracy_score(y_pred,y_val)

In [ ]:
confusion_matrix(y_pred,y_val)

In [ ]:
data_test